# Lesson 2 - Session 1 - String is all you need

### Learning Objectives:

1. Understand that in Bash, all variables are treated as strings by default.
2. Learn how to declare and initialize variables in Bash.
3. Explore how to perform arithmetic operations in Bash and the need for explicit type conversion.
4. Practice using declare and let for type conversion and arithmetic operations.
5. Understand the implications of string-based operations in scripting and how to handle them.

## 1. All variables are treated as strings by default

By default, all variables in Bash are treated as strings. Specifically, all variables in Bash are **untyped**, and if a variable contains only digits, it can be treated as an integer depending on the context. This can be confusing because it differs from other programming languages. Let's see how it works.

In [27]:
%%bash

a=5
b=3
output=$a+$b
echo "result without expr: $output"

result without expr: 5+3


As you can see, you cannot add or multiply the a and b variables just by declaring them and using `$a+$b`. All elements, including `+`, are treated as strings. However, you can perform integer calculations using the following three methods.

## 2. Calculation 

One way to perform integer operations in Bash is by using double brackets `$(())`. By declaring digit-only variables and using operators and variables in the `$((a + b))` format, you can perform arithmetic operations in Bash.

Another way to perform integer operations in Bash is by using the expr command. By declaring digit-only variables and using operators and variables in the expr `$a + $b` format, you can perform arithmetic operations in Bash.

You can also use the let syntax to perform arithmetic operations in Bash. By declaring digit-only variables and using operators and variables in the `let c=$a+$b` format, you can perform arithmetic operations in Bash.

Operators that can be used in arithmetic operations are `+`, `-`, `*`, `/`, `%`, `**`, and `+=`, `-=`, `*=`, `/=`, `%=`, `**=` for assignment operations.


In [47]:
%%bash

a=10
b=5
double_brackets=$(($a + $b))
expr_outputs=$(expr $a \* $b)
divide=$((a / b))
square=$((a ** b))
let plus_equal=$a*$b

echo "result in (()): $double_brackets"
echo "result in expr $expr_outputs"
echo "result in divide: $divide"
echo "result of square: $square"
echo "result of plus equal: $plus_equal"

result in (()): 15
result in expr 50
result in divide: 2
result of square: 100000
result of plus equal: 50


One thing you should know is that only integers can be used in arithmetic operations. If you try to use floating-point numbers, you will get an error like below. You should use `bc` command to calculate floating-point numbers.

In [31]:
%%bash

a=5.3
b=3.2

double_brackets=$((a + b))
expr_outputs=$(expr $a + $b)

echo "result in (()): $double_brackets"
echo "result in expr $expr_outputs"

bash: line 5: 5.3: syntax error: invalid arithmetic operator (error token is ".3")
expr: not a decimal number: '5.3'


result in (()): 
result in expr 


As you can see, floating points are treated as string, therefore error has raised in the script. but if you use bc, you can calculate floating point numbers.

In [8]:
%%bash

num1=10.3
num2=3.2

product=$(echo "$num1 * $num2" | bc)
quotient=$(echo "scale=2; $num1 / $num2" | bc)

echo "Product: $product"
echo "Quotient: $quotient"

Product: 32.9
Quotient: 3.21


Unlike other programming languages, Bash does not support floating-point arithmetic. To perform such calculations, you need to use the bc command, a command-line calculator that handles floating-point arithmetic.

While multiplication can be performed using the `*` operator, division requires the scale option in bc. The scale option specifies the number of decimal places in the result. For example, setting `scale=2` ensures that the quotient is rounded to two decimal places. Without this option, division results are truncated to the nearest integer.

In [12]:
%%bash

num1=11
num2=3

quotient=$(echo "$num1 / $num2" | bc)
echo $quotient

3


## 3. Compare values

As I've described above, all values are treated as strings in Bash. Therefore, when you compare two variables, you should be careful. If you compare two variables, Bash will compare them as strings.

You can use -eq for numeric comparison and = for string comparison. If you use -eq for string comparison, It will get error like below.

In [17]:
%%bash

if [ "hi" -eq "bye" ]; then
    echo "true"
fi

bash: line 2: [: hi: integer expression expected


However, it does not mean that you cannot compare numbers and string that contains only digits. In this situation, you can use -eq for comparison because it treat variables as integers. You can see the example below.

In [15]:
%%bash

int=10
str="10"

if [ "$int" -eq "$str" ]; then
  echo "Integer and string are equal"
else
  echo "Integer and string are not equal"
fi


Integer and string are equal
